In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.CommandLineAuth()
drive = GoogleDrive(gauth)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=198855760514-cttaak3avfg6568amn6d04dv7p7tfbdt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



Enter verification code:  4/1AX4XfWhq45XVWgVywX3v6CAXbJtZbUxslpAtbdn63o-W3JuSHJosprmdycA


Authentication successful.


In [16]:
def interval_column(df):
    id_list = df.eye_id.unique()
    df.admission_date = pd.to_datetime(df.admission_date)
    frames = []
    for eye in id_list:
        interval_list = [0]
        pdf = df[df.eye_id == eye]
        pdf.sort_values(by=['admission_date'], inplace=True)
        for i in range(len(pdf)-1):
            interval = np.round((pdf.admission_date.iloc[i+1] - pdf.admission_date.iloc[i]).days / 7)
            interval_list.append(interval)
        pdf['Interval'] = interval_list
        frames.append(pdf)
    return pd.concat(frames)

def year_clean(df):
    id_list = df.eye_id.unique()
    df.admission_date = pd.to_datetime(df.admission_date)
    frames = []
    cutoff = pd.to_datetime('2016-01-01')
    for eye in id_list:
        pdf = df[df.eye_id == eye]
        pdf.sort_values(by=['admission_date'], inplace=True)
        if pdf.admission_date.iloc[0] > cutoff:
            frames.append(pdf)
    return pd.concat(frames)

In [25]:
class DataSeparator:
    """
    A class used to segment the dataset in different ways.

    ...

    Attributes
    ----------
    df_list : str
        a formatted string to determine subset of patients being analysed
        options: 
            - 'drugs': segment patients by those on single vs mult drugs
            - 'improved': segment patients by those who improved vs didn't
            - 'switched': segment patients by switches (L to E, E to L only)
            - 'all': segment by all of the above

    Methods
    -------
    get_df(self, drop_drug_na=False)
        Retrieves the specific dataset from the path.
        'drop_drug_na=True' will remove NAs and Brolucizumab from 'Drug' column.
    get_dataframes(self)
        Produces a list of dataframes based on df_list segmentation query.
    drug_df_separator(self)
        Segments the dataset by patients on single vs mult drugs.
    improved_df_separator(self)
        Segments the dataset by patients who improved vs didn't.
    switch_df_separator(self)
        Segments the dataset by patients who switched.
        Lucentis to Eylea, Eylea to Lucentis switches only.
    """
    
    def __init__(self, df_list):
        self.df_list = df_list
        
    def get_df(self, drop_drug_na=False):
        """
        Retrieves the specific dataset from the path.
        Input: Bool.
        Output: Pandas dataframe.
        """
        df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/eric2016.csv')
        df['admission_date'] = pd.to_datetime(df['admission_date'])
        if drop_drug_na:
            df = df.dropna(subset=['Drug'])
            df = df[df['Drug'] != 'nil']
            df = df[df['Drug'] != 'None']
            df = df[df['Drug'] != 'Brolucizumab']
        return df
    
    def get_dataframes(self):
        """
        Produces a list of dataframes based on df_list segmentation query.
        Input: self.
        Output: [Pandas dataframe].
        """
        if self.df_list == 'drugs':
            lucentis_df, eylea_df, mult_df = self.drug_df_separator()
            return [lucentis_df, eylea_df, mult_df]
        elif self.df_list == 'switched':
            luc_eyl, eyl_luc = self.switch_df_separator()
            luc_eyl
            return [luc_eyl, eyl_luc]
        elif self.df_list == 'improved':
            improved, not_improved, overall = self.improved_df_separator()
            return [improved, not_improved, overall]
        elif self.df_list == 'all':
            lucentis_df, eylea_df, mult_df = self.drug_df_separator()
            improved, not_improved, overall = self.improved_df_separator()
            luc_eyl, eyl_luc = self.switch_df_separator()
            return [lucentis_df, eylea_df, mult_df, improved, not_improved, overall, luc_eyl, eyl_luc]
        
    def drug_df_separator(self):
        """
        Produces a list of dataframes based single vs mult drugs.
        Input: self.
        Output: [Pandas dataframe].
        """
        df = self.get_df(drop_drug_na=True)
        df = df.dropna(subset=['Drug'])
        lucentis, eylea, avastin, mult = [], [], [], []
        drug_list = ['Lucentis', 'Eylea', 'Avastin']
        id_list = df['eye_id'].unique()
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            drugs = pdf['Drug'].unique()
            if len(drugs) == 1:
                if drugs[0] == 'Lucentis':
                    lucentis.append(pdf)
                elif drugs[0] == 'Eylea':
                    eylea.append(pdf)
                else:
                    avastin.append(pdf)
            else:
                mult.append(pdf)
        return pd.concat(lucentis), pd.concat(eylea), pd.concat(mult)
    

    def improved_df_separator(self):
        """
        Produces a list of dataframes based on improvement.
        Input: self.
        Output: [Pandas dataframe].
        """
        df = self.get_df()
        improved, not_improved = [], []
        id_list = df['eye_id'].unique()
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            pdf.sort_values(by=['admission_date'], inplace=True)
            vision = pdf['visual_acuity'].to_list()
            if max(vision) == vision[0]:
                not_improved.append(pdf)
            else:
                improved.append(pdf)
        return pd.concat(improved), pd.concat(not_improved), df
    
    def switch_df_separator(self):
        """
        Produces a list of dataframes based on switches.
        Input: self.
        Output: [Pandas dataframe].
        """
        df = self.get_df(drop_drug_na=True)
        df = df[df['Drug'] != 'Avastin']
        luc_eyl, eyl_luc = [], []
        drug_list = ['Lucentis', 'Eylea']
        id_list = df['eye_id'].unique()
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            drugs = pdf['Drug'].unique()
            if len(drugs) > 1:
                if drugs[0] == 'Lucentis' and drugs[1] == 'Eylea':
                    luc_eyl.append(pdf)
                elif drugs[0] == 'Eylea' and drugs[1] == 'Lucentis':
                    eyl_luc.append(pdf)
            else:
                pass
        return pd.concat(luc_eyl), pd.concat(eyl_luc)
    
class InitialMetrics(DataSeparator):
    
    def __init__(self, df_list):
        self.df_list = df_list
    
    def study_period(self, df):
        """
        Returns the study period in question.
        Input: Pandas dataframe.
        Output: datetime object x 2.
        """
        dates = df['admission_date']
        return dates.min(), dates.max()
    
    def num_patients(self, df):
        """
        Returns the number of patients treated.
        """
        return len(df['ur'].unique())
    
    def num_eyes(self, df):
        """
        Returns the number of eyes treated.
        """
        return len(df['eye_id'].unique())
    
    def total_visits(self, df):
        """
        Returns the number of total visits.
        """
        return len(df)
    
    def initiation_drug(self, df):
        '''
        Input: Pandas dataframe of all patients.
        Output: mean initiation drugs.
        '''
        id_list = df['eye_id'].unique()
        lst, names = [], ['Lucentis', 'Eylea', 'Avastin']
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            pdf.dropna(subset=['Drug'], inplace=True)
            df = df[df['Drug'] != 'nil']
            df = df[df['Drug'] != 'None']
            if len(pdf['Drug']) != 0 and pdf['Drug'].iloc[0] in names:
                lst.append(pdf['Drug'].iloc[0])
        luc_int = np.round(lst.count('Lucentis') / len(lst), 3)
        eyl_int = np.round(lst.count('Eylea') / len(lst), 3)
        av_int = np.round(lst.count('Avastin') / len(lst), 3)
        return luc_int, eyl_int, av_int

    def irf_extred(self):
        # this function counts the irf values when extending and reducing
        df = self.interval_df()
        df.dropna(subset=['irf'], inplace=True)
        df = df[df['irf'] < 1000]
        id_list = df['eye_id'].unique()
        irf_extend = []
        irf_reduce = []
        irf_maintain = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            for i in range(len(pdf)-1):
                if pdf['Interval'].iloc[i+1] > pdf['Interval'].iloc[i]:
                    irf_extend.append(pdf['irf'].iloc[i+1])
                elif pdf['Interval'].iloc[i+1] < pdf['Interval'].iloc[i]:
                    irf_reduce.append(pdf['irf'].iloc[i+1])    
                else:
                    irf_maintain.append(pdf['irf'].iloc[i+1])   
        return np.mean(irf_extend), np.mean(irf_reduce), np.mean(irf_maintain)
    
    # this function counts the srf values when extending and reducing
    def srf_extred(self):
        df = self.interval_df()
        df.dropna(subset=['srf'], inplace=True)
        df = df[df['srf'] < 1000]
        id_list = df['eye_id'].unique()
        srf_extend = []
        srf_reduce = []
        srf_maintain = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            for i in range(len(pdf)-1):
                if pdf['Interval'].iloc[i+1] > pdf['Interval'].iloc[i]:
                    srf_extend.append(pdf['srf'].iloc[i+1])
                elif pdf['Interval'].iloc[i+1] < pdf['Interval'].iloc[i]:
                    srf_reduce.append(pdf['srf'].iloc[i+1])    
                else:
                    srf_maintain.append(pdf['srf'].iloc[i+1])   
        return np.mean(srf_extend), np.mean(srf_reduce), np.mean(srf_maintain)
    
            
    def results_table(self):
        """
        Returns a dataframe of results for all initial metrics.
        Input: self.
        Output: Pandas dataframe.
        """
        df = DataSeparator.get_df(self)
        earliest, latest = self.study_period(df)
        luc_int, eyl_int, av_int = self.initiation_drug(df)
        dict = {'Start': earliest, 'End': latest, 'Num_Patients': self.num_patients(df), 'Num_Eyes': self.num_eyes(df),
               'TotalVisits': self.total_visits(df), 'LucentisInit%': luc_int*100, 'EyleaInit%': eyl_int*100, 
                'AvastinInit%': av_int*100} 
        return pd.DataFrame(dict, index=[0])
    
    def upload_results_table(self):
        dataframe = self.results_table()
        dataframe.to_csv('/home/jupyter/charliemacuject/pharma_reports/initialmetrics.csv')
        

class Vision(DataSeparator):
    """
    A class used to produce common visual metrics.

    ...

    Attributes
    ----------
    df_list : str
        a formatted string to determine subset of patients being analysed
        options: 
            - 'drugs': segment patients by those on single vs mult drugs
            - 'improved': segment patients by those who improved vs didn't
            - 'switched': segment patients by switches (L to E, E to L only)
            - 'all': segment by all of the above

    Methods
    -------
    pvi(self, dataset)
        Produces a list of patient PVIs from dataset.
    tpvi(self, dataset)
        Produces a list of patient TPVIs from dataset.
    vlp(self, dataset)
        Produces a list of patient VLPs from dataset.
    ovc(self, dataset)
        Produces a list of patient OVCs from dataset.
    vision_weeks(self, weeks)
        Calculates the mean vision for all patients at a specific visit number.
    anchor(self, weeks)
        Produces a dataframe of mean vision over the first year, including trial results.
    list_results(self, dataset)
        Produces a list of above visual metrics (PVI, TPVI, VLP, OVC) for given dataset.
    list_sds(self, dataset)
        Produces a list of the 95% CI bounds for all visual metrics, for given dataset.
    print_results(self)
        Displays results for all visual metrics.
    results_table(self)
        Returns a Pandas dataframe of all visual metrics.
    """
    
    # df_list can be either drugs, improved, switched or all
    def __init__(self, df_list):
        self.df_list = df_list
    
    def pvi(self, df):
        """
        Produces a list of patient PVIs from dataset.
        Peak Visual Improvement (PVI) is defined as max vision minus initial vision.
        Input: Pandas dataframe.
        Output: list of LogMAR letters.
        """
        id_list = df['eye_id'].unique()
        pvi_list = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            visions = pdf['visual_acuity'].to_list()
            if len(visions) > 0:
                pvi_pdf = max(visions) - visions[0]
                pvi_list.append(pvi_pdf)
        return pvi_list
    
    def tpvi(self, df):
        """
        Produces a list of patient TPVIs from dataset.
        Time to Peak Visual Improvement (TPVI) is defined in days.
        Input: Pandas dataframe.
        Output: list of days.
        """
        id_list = df['eye_id'].unique()
        tpvi_list = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            dates = pdf['admission_date'].to_list()
            initial_date = dates[0]
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            visions = pdf['visual_acuity'].to_list()
            if len(visions):
                ndf = pdf[pdf['visual_acuity'] == max(visions)]
                new_dates = ndf['admission_date'].to_list()
                time = (new_dates[0] - initial_date).days
                tpvi_list.append(time)
        return tpvi_list
    
    def ovc(self, df):
        """
        Produces a list of patient OVCs from dataset.
        Overall Visual Change (OVC) is defined as last vision minus initial vision.
        Input: Pandas dataframe.
        Output: list of LogMAR letters.
        """
        id_list = df['eye_id'].unique()
        ovc_list = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            visions = pdf['visual_acuity'].to_list()
            if len(visions) > 0:
                ovc_list.append(visions[-1] - visions[0])
        return ovc_list
    
    def vlp(self, df):
        """
        Produces a list of patient VLPs from dataset.
        Vision Loss from Peak (VLP) is defined as max vision minus last vision.
        Input: Pandas dataframe.
        Output: list of LogMAR letters.
        """
        id_list = df['eye_id'].unique()
        vlp_list = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            visions = pdf['visual_acuity'].to_list()
            if len(visions) > 0:
                recent = visions[-1]
                max_vision = max(visions)
                vlp_list.append(max_vision - recent)
        return vlp_list
    
    def month_column(self, pdf):
        pdf.sort_values(by=['admission_date'], inplace=True)
        visions = pdf['visual_acuity'].to_list()
        initial_date = pdf.admission_date.iloc[0]
        months_list = [0]
        for i in range(len(pdf)-1):
            months = round((pdf.admission_date.iloc[i] - initial_date).days / 28)
            months_list.append(months)
        pdf['month_since_start'] = months_list
        return pdf
    
    def vision_month(self, month):
        """
        Produces average vision at a certain visit after initiation.
        Input: dataframe, number of visits after initiation (int)
        Output: mean visual acuity of all patients at that visit
        """
        df = DataSeparator.get_df(self)
        id_list = df['eye_id'].unique()
        df.dropna(subset=['visual_acuity'], inplace=True)
        lst = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            pdf = self.month_column(pdf)
            visions = pdf['visual_acuity'].to_list()
            for i in range(len(pdf)):
                if pdf.month_since_start.iloc[i] == month:
                    lst.append(visions[i] - visions[0])
        return np.round(np.mean(lst), 2)
    
    def anchor(self):
        """
        Produces a dataframe of mean vision over the first year, including trial results.
        Also produces the standard ANCHOR figures for the same period.
        Input: Pandas dataframe (all patients).
        Output: Pandas dataframe.
        """
        df = DataSeparator.get_df(self)
        anchor_list = [0]
        months = [x for x in range(26)]
        rani5 = [0.0, 4.6, 8.4, 9.8, 10, 9.9, 10.2, 10.6, 10.2, 10.9, 11.4,
                 10.9, 11.1, 11.3, 11.0, 11.4, 11.4, 11.4, 11.0, 11.1, 
                 11.3, 11.0, 10.9, 11.0, 11.0, 10.7]
        rani3 = [0.0, 2.9, 5.9, 6.4, 6.8, 7.2, 7.4, 7.9, 8.2, 7.7, 8.1, 
                 7.8, 8.6, 8.5, 8.4, 8.6, 9.4, 9.0, 9.0, 9.2, 9.0,
                 9.0, 8.4, 8.1, 8.0, 8.1]
        vert = [0.0, 3.9, 0.5, -1.8, -2.5, -3.1, -4.1, -5.6, -6.8, -7.1, -7.1,
                -8.3, -9.1, -9.5, -10.8, -10.7, -10.4, -10.8, -11.0, -11.2,
                -11.4, -11.1, -10.7, -11.1, -10.2, -9.8]
        for i in range(25):
            anchor_list.append(self.vision_month(i))
        dict = {'Month': months, 'Doctor_Mean': anchor_list, 'Ranibizumab_0.5mg': rani5,
               'Ranibizumab_0.3mg': rani3, 'Verteporfin': vert} 
        df = pd.DataFrame(dict)
        df.set_index('Month', inplace=True)
        return df
    
    def list_results(self, df):
        """
        Produces a list of the visual metrics defined (PVI, TPVI, VLP, OVC).
        Input: Pandas dataframe.
        Output: float x 4.
        """
        pvi_list, tpvi_list, vlp_list, ovc_list = self.pvi(df), self.tpvi(df), self.vlp(df), self.ovc(df)
        return np.mean(pvi_list), np.mean(tpvi_list), np.mean(vlp_list), np.mean(ovc_list)
    
    def list_sds(self, df):
        """
        Produces a list of the 95% CI bounds for all visual metrics.
        Input: Pandas dataframe.
        Output: float x 4
        """
        pvi_list, tpvi_list, vlp_list, ovc_list = self.pvi(df), self.tpvi(df), self.vlp(df), self.ovc(df)
        lst = [pvi_list, tpvi_list, vlp_list, ovc_list]
        to_return = []
        for item in lst:
            ci = stats.t.interval(alpha=0.95, df=len(item)-1, loc=np.mean(item), scale=stats.sem(item))
            std = np.mean(item) - ci[0]
            to_return.append(std)
        return np.round(to_return[0], 2), np.round(to_return[1], 2), np.round(to_return[2], 2), np.round(to_return[3], 2)

    def print_results(self):
        """
        Displays the results for all visual metrics.
        Input: self.
        Output: float x 4
        """
        dfs = DataSeparator.get_dataframes(self)
        lst = []
        if self.df_list == 'drugs':
            print("Displaying results for Lucentis, Eylea, and Multiple Drugs.")
        elif self.df_list == 'improved':
            print("Displaying results for Improved, Not_Improved and Overall.")
        elif self.df_list == 'switched':
            print("Displaying results for patients who switched from Lucentis to Eylea, and Eylea to Lucentis.")
        else:
            print("Displaying results for Lucentis, Eylea, Multiple Drugs, Improved, Didn't Improve, Lucentis to Eylea, Eylea to Lucentis.")
        for dataframe in dfs:
            pvi, tpvi, vlp, ovc = self.list_results(dataframe)
            pvi_sd, tpvi_sd, vlp_sd, ovc_sd = self.list_sds(dataframe)
            print("  ")
            print("PVI = {} letters, CI = +/-{}".format(np.round(pvi, 2), pvi_sd))
            print("TPVI = {} days, CI = +/-{}".format(np.round(tpvi), tpvi_sd))
            print("VLP = {} letters, CI = +/-{}".format(np.round(vlp, 2), vlp_sd))
            print("OVC = {} letters, CI = +/-{}".format(np.round(ovc, 2), ovc_sd))
            print("  ")
            
    def results_table(self):
        """
        Returns a dataframe of results for all visual metrics.
        Input: self.
        Output: Pandas dataframe.
        """
        names = ['Lucentis', 'Eylea', 'Multiple', 'Improved', 'Not_Improved', 'Overall', 'Luc_to_Eyl', 'Eyl_to_Luc']
        df_lst = DataSeparator.get_dataframes(self)
        pvi, tpvi, vlp, ovc, pvi_sd, tpvi_sd, vlp_sd, ovc_sd = [], [], [], [], [], [], [], []
        for i in range(len(df_lst)):
            dataframe = df_lst[i]
            name = names[i]
            Pvi, Tpvi, Vlp, Ovc = self.list_results(dataframe)
            Pvi_sd, Tpvi_sd, Vlp_sd, Ovc_sd = self.list_sds(dataframe)
            pvi.append(Pvi), tpvi.append(Tpvi), vlp.append(Vlp), ovc.append(Ovc)
            pvi_sd.append(Pvi_sd), tpvi_sd.append(Tpvi_sd), vlp_sd.append(Vlp_sd), ovc_sd.append(Ovc_sd)
        dict = {'Drug': names, 'PVI': pvi, 'PVI_sd': pvi_sd, 'TPVI': tpvi, 'TPVI_sd': tpvi_sd,
                'VLP': vlp, 'VLP_sd': vlp_sd, 'OVC': ovc, 'OVC_sd': ovc_sd} 
        df = pd.DataFrame(dict)
        df.set_index('Drug', inplace=True)
        return df
    
    def upload_results_table(self):
        dataframe = self.results_table()
        dataframe.to_csv('/home/jupyter/charliemacuject/pharma_reports/visual_metrics.csv')
        anc = self.anchor()
        anc.to_csv('/home/jupyter/charliemacuject/pharma_reports/anchor.csv')
        
        
class Upload:
    """
    A class used to upload the required dataframes to Google Drive.

    ...

    Attributes
    ----------
    folder : str
        a formatted string to determine the folder to upload to
        options: 'pharma_reports', 'research'
    dr_name : str
        a formatted string of audited doctor, used to create folder in drive
    file_list : list
        a list of files to upload to Drive

    Methods
    -------
    set_folder(self)
        Determines the path ID of the folder to upload to.
    view_files(self)
        View all folders and file in the Google Drive.
    create_folder(self)
        Create folder on Google Drive.
    upload_files(self)
        Upload the specified file of the object.
    """
    
    def __init__(self, folder, dr_name, file_list):
        self.folder = folder
        self.dr_name = dr_name
        self.file_list = file_list
        
    def retrieve_files(self):
        """
        Returns a list of files in the main directory to upload.
        """
        if self.file_list == 'all':
            return ['initialmetrics.csv','visual_metrics.csv', 'anchor.csv']
        
    def set_folder(self):
        """
        Determines the path ID of the folder to upload to.
        Input: self.
        Output: folder_id (PyDrive object)
        """
        if self.folder == 'pharma_reports':
            folder_id = '1INvWwtJhtiD4vqCEpehd4RT06I1-50pU'
        elif self.folder == 'research':
            folder_id = '1GuUeRc7Zqe5GdM1dmJOUF6VPW5NklH-R'
        else:
            folder_id = '1INvWwtJhtiD4vqCEpehd4RT06I1-50pU'
        return folder_id
    
    def view_files(self):
        """
        View all folders and file in the Google Drive.
        """
        fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
        for file in fileList:
            print('Title: %s, ID: %s' % (file['title'], file['id']))
            
    def create_folder(self):
        """ 
        Create folder on Google Drive.
        """
        folder_name, parent_folder_id = self.dr_name, self.set_folder()
        folder_metadata = {'title': folder_name,
                           # Define the file type as folder
                           'mimeType': 'application/vnd.google-apps.folder',
                           # ID of the parent folder        
                           'parents': [{"kind": "drive#fileLink", "id": parent_folder_id}]}
        folder = drive.CreateFile(folder_metadata)
        folder.Upload()
        # Return folder informations
        return folder['id']
                
    def upload_files(self):
        """
        Upload the specified file of the object.
        """
        folder_id = self.create_folder()
        upload_file_list = self.retrieve_files()
        for upload_file in upload_file_list:
            gfile = drive.CreateFile({'parents': [{'id': folder_id}]})
            # Read file and set it as the content of this instance.
            gfile.SetContentFile(upload_file)
            gfile.Upload() # Upload the file.

In [40]:
def generate_audit(dr_name, folder='pharma_reports', file_list='all', df_list='all'):
    #initialmetrics = InitialMetrics(df_list)
    #initialmetrics.upload_results_table()
    visual_metrics = Vision(df_list)
    visual_metrics.upload_results_table()
    
    to_upload = Upload(folder, dr_name, file_list)
    to_upload.upload_files()

In [41]:
generate_audit('EricMayer')